In [ ]:
from src.genetic import Fitness
from src.utils import Database, DBKey, divide
import math, json, os, glob


problem_map = {
    'ICL1701': '1',
    'TRMAG': '2',
    'MAXSCRAB': '3',
    'LPAIR': '4',
    'RUMBLING': '5',
    'TREEROOT': '6',
    'MINORPATH': '7'
}
fitness = Fitness()
stars = {obj:{} for obj in fitness.objectives}
dataset = 'data'
experiments = glob.glob(f'{dataset}/**/experiment.json', recursive=True)
for exp in experiments:
    exp_db = Database(exp)
    for data in exp_db.all():
        problem = problem_map[data[DBKey.problem]]
        approach = data[DBKey.approach]
        for obj in fitness.objectives:
            stars[obj].setdefault(problem, {})
            stars[obj][problem][approach] = data[obj]

stars.update({'minimal changes': stars['similarity']})

In [ ]:
overall_scores = {obj:{} for obj in fitness.objectives}
for obj in fitness.objectives:
    for problem in stars[obj]:
        scores = stars[obj][problem]
        for approach in scores:
            overall_scores[obj].setdefault(approach, 0)
            overall_scores[obj][approach] += scores[approach]

overall_scores

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

approach_map = {
    'zero': 'zero-shot',
    'random': 'few-shot(random)',
    'optimal': 'few-shot(optimal)',
}
color_map = {
    'zero': 'red',
    'random': 'green',
    'optimal': 'blue',
}

sorted_stars = sorted(stars['minimal changes'])
fig, axes = plt.subplots(1, 3, figsize=(18, 6), sharex=True, sharey=False)
for ax, obj in zip(axes, fitness.objectives):
    if obj == 'similarity':
        obj = 'minimal changes'
    for i, approach in enumerate(approach_map.keys(), 1):
        data = [stars[obj][star][approach] for star in sorted_stars]
        ax.fill_between(sorted_stars, 
                        data, 
                        label=approach_map[approach], 
                        color=color_map[approach], 
                        alpha=0.1 * i)

    ax.set_title(obj.title())
    ax.set_xlabel('Levels')
    ax.set_ylabel('Fitness Score')
    ax.legend()
    ax.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
import json
import matplotlib.pyplot as plt
from collections import defaultdict

# JSON 파일 로드
with open('data/ICL1701/feedbacks/optimal/results.json') as f:
    data = json.load(f)

# _default 내부 데이터
records = data['_default']

# fn 그룹으로 묶기
grouped = defaultdict(list)
for entry in records.values():
    for key in entry:
        if key.startswith("f"):
            grouped[key].append({
                "Repair Rate": entry["Repair Rate"],
                "Relative Patch Size": entry["Relative Patch Size"],
                "AVG Time Taken(sec)": entry["AVG Time Taken(sec)"],
            })

# 평균 계산
averages = {}
for fn, entries in grouped.items():
    avg_repair = sum(e["Repair Rate"] for e in entries) / len(entries)
    avg_patch_size = sum(e["Relative Patch Size"] for e in entries) / len(entries)
    avg_time = sum(e["AVG Time Taken(sec)"] for e in entries) / len(entries)
    averages[fn] = {
        "Repair Rate": avg_repair,
        "Relative Patch Size": avg_patch_size,
        "AVG Time Taken(sec)": avg_time
    }

# 그래프 그리기
metrics = ["Repair Rate", "Relative Patch Size", "AVG Time Taken(sec)"]
for metric in metrics:
    plt.figure(figsize=(8, 5))
    plt.bar(averages.keys(), [v[metric] for v in averages.values()])
    plt.title(f'Average {metric} by fitness function')
    plt.xlabel('fitness function')
    plt.ylabel(metric)
    plt.ylim(0, max(v[metric] for v in averages.values()) * 1.1)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()


In [ ]:
from src.utils import Database

dataset = "data/dataset.db"
dataset_db = Database(dataset)
problem_tb = dataset_db.get_table('problem')
experiments_tb = dataset_db.create_table('experiments')

for i in range(1, 13):
    for expeiment in experiments_tb.find(problemId=i):
    

In [ ]:
import json

with open('contest.json', 'r') as f:
    contest = json.load(f)

result = contest['result']

In [ ]:
data = {i:[] for i in range(1, 6)}
for contest in result:
    if 'startTimeSeconds' in contest.keys() \
        and 'difficulty' in contest.keys():
        if contest['startTimeSeconds'] > 1717167600:
            data[contest['difficulty']].append(contest)

In [ ]:
from src.utils import Randoms
for difficulty, contests in data.items():
    print(f"Difficulty {difficulty}: {len(contests)} contests")
    contest = Randoms.choice(contests)
    print(contest['id'], contest['name'])
    

In [ ]:
import json

with open('data/1/logs_hvpe_1.json', 'r') as f:
    logs = json.load(f)


for generation, results in logs.items():
    threshold = 0
    for b_id, solutions in results.items():
        for p_id, scores in solutions.items():
            scores = list(scores.values())
            min_score = min(scores)
            threshold = max(threshold, min_score)
    print(f"Generation {generation}: Threshold = {threshold}")
            
                
    

In [ ]:
from src.utils import Database

overall = {}
db = Database('data2/dataset.db')
experiments = db.get_table('experiments')

for exp in experiments.all():
    problemId = exp['problemId']
    generations = exp['generations']
    selection = exp['selection']
    overall.setdefault(problemId, {})
    overall[problemId].setdefault(generations, {})
    overall[problemId][generations].setdefault(selection, {})
    
    rr = exp['avg_rr']
    rps = exp['avg_rps']
    att = exp['avg_att']
    f1 = exp['f1']
    f2 = exp['f2']
    f3 = exp['f3']
    f4 = exp['f4']
    f5 = exp['f5']
    f6 = exp['f6']
    f7 = exp['f7']
    overall[problemId][generations][selection] = {
        'rr': rr, 'rps': rps, 'att': att,
        'fitness': {'f1': f1, 'f2': f2, 'f3': f3,
        'f4': f4, 'f5': f5, 'f6': f6, 'f7': f7}
    }

In [ ]:
import json
import math
import os
import warnings
warnings.filterwarnings("ignore")

from src.utils import TinyDatabase, DBKey, ETC, TED
from src.execution import Tester
from src.genetic import Fitness

def ratio(numerator: float, denominator: float) -> float:
    if numerator == 0 or denominator == 0:
        return 0.0
    value = ETC.divide(numerator, denominator)
    abs_value = abs(value)
    decimal_places = max(3, -int(math.floor(math.log10(abs_value))))
    return round(value, decimal_places)

problemId = 1
db = TinyDatabase(f'data2/{str(problemId)}/dataset.json')
problem = db.get_data_from_table('problem')
title = problem['title']
description = problem['description']
references = db.get_data_from_table('references')
buggys = db.get_data_from_table('buggys')
testcases = db.table('testcases').all()
programs = references | buggys
Tester.init_globals(testcases, 1, 500000, title, description)

for b_id, buggy in references.items():
    results = Tester.trace(buggy)
    test_hist = Tester.get_exec_hist(results)
    print(test_hist)
    break
# fitness = Fitness()

# for b_id, buggy in buggys.items():
#     for obj in fitness.objectives:
#         fitness.OBJ_FUNC_MAP[obj](buggy, references)
  
# with open('data2/1/best_solution.json', 'r') as f:
#     solutions = json.load(f)

# fixed = len(solutions)
# tot_rps = 0
# overall_objectives = {obj:0 for obj in Fitness.OBJECTIVES}
# for b_id, solution in solutions.items():
#     buggy = buggys[b_id]
#     patch = solution['patch']
#     scores = fitness.run(buggy, patch)
#     # tot_rps += TED.relative_patch_size(buggy, patch)
#     for obj in Fitness.OBJECTIVES:
#         overall_objectives[obj] += scores[obj]
    
# exp = {
#     DBKey.problemId: problemId,
#     DBKey.title: title,
#     DBKey.fitness: 'f1f2f3f4f5f6f7',
#     DBKey.generations: 10,
#     DBKey.pop_size: 10,
#     DBKey.selection: 'Mentored',
#     DBKey.trials: 10,
#     DBKey.b_progs: len(buggys),
#     DBKey.a_sol: 64,
#     DBKey.a_rr: 0.33,
#     DBKey.a_rps: 0.1,
#     DBKey.a_att: 7.3,
# }
# exp.update(overall_objectives)
# print(exp)

# rr = exp['avg_rr']
# rps = exp['avg_rps']
# att = exp['avg_att']
# f1 = exp['f1']
# f2 = exp['f2']
# f3 = exp['f3']
# f4 = exp['f4']
# f5 = exp['f5']
# f6 = exp['f6']
# f7 = exp['f7']
    
# overall[1][10]['Mentored'] = {
#     'rr': rr, 'rps': rps, 'att': att,
#     'fitness': {'f1': f1, 'f2': f2, 'f3': f3,
#     'f4': f4, 'f5': f5, 'f6': f6, 'f7': f7}
# }

num_int,max_ops=map(int,input().split())
arr=list(map(int,input().split()))
carry=0
count=0
lst_ind=[]
ops_ind1=0
for ops_ind2 in range(num_int):
    val=arr[ops_ind2]
    if val<0:
        bit=0
    else:
        bit=1
    bit_sum=bit+carry
    if bit_sum==2 or bit_sum==0:
        count+=1
        carry=1
        lst_ind.append(ops_ind2+1)
    elif bit_sum==1:
        carry=0
    if count>max_ops:
        print(-1)
        break
else:
    print(count)
    flag=0
    for val2 in lst_ind:
        print(val2,end=" ")
        flag=1
    if flag==1:
        print()
{1: [], 2: [], 3: [], 4: [], 5: []}


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict


TARGET_GENERATION = 10
methods = ['Refactory', 'Mentored', 'MooRepair']

# 데이터 전처리
problemIds = sorted(overall.keys())
num_problems = len(problemIds)

# subplot 설정 (2행 4열)
fig, axes = plt.subplots(2, 4, figsize=(20, 10))
axes = axes.flatten()

# 각 problemId별 subplot 생성
for i, problemId in enumerate(problemIds):
    ax = axes[i]
    
    # TARGET_GENERATION이 해당 problemId에 존재하는지 확인
    if TARGET_GENERATION not in overall[problemId]:
        ax.set_title(f'Problem {problemId} (No data for gen {TARGET_GENERATION})')
        ax.text(0.5, 0.5, f'No data for\ngeneration {TARGET_GENERATION}', 
                ha='center', va='center', transform=ax.transAxes)
        continue
    
    # 각 selection별로 선 그리기
    for j, selection in enumerate(methods):
        if selection not in overall[problemId][TARGET_GENERATION]:
            continue
        fitness_data = overall[problemId][TARGET_GENERATION][selection]['fitness']
        fitness_values = [
            fitness_data['f1'], fitness_data['f2'], fitness_data['f3'],
            fitness_data['f4'], fitness_data['f5'], fitness_data['f6'],
            fitness_data['f7']
        ]
        
        # 선 그리기
        x_labels = ['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7']
        ax.plot(x_labels, fitness_values, marker='o', label=selection, alpha=0.4)
    
    ax.set_title(f'Problem {problemId} (Gen {TARGET_GENERATION})')
    ax.set_xlabel('Objective Functions')
    ax.set_ylabel('Fitness Value')
    ax.legend()

# 마지막 subplot에 전체 평균 그리기
if num_problems < len(axes):
    ax = axes[num_problems]
    
    # 모든 problemId에서 TARGET_GENERATION의 selection별 평균 계산
    selection_averages = defaultdict(list)
    
    for problemId in problemIds:
        if TARGET_GENERATION in overall[problemId]:
            for selection in methods:
                fitness_data = overall[problemId][TARGET_GENERATION][selection]['fitness']
                fitness_values = [
                    fitness_data['f1'], fitness_data['f2'], fitness_data['f3'],
                    fitness_data['f4'], fitness_data['f5'], fitness_data['f6'],
                    fitness_data['f7']
                ]
                selection_averages[selection].append(fitness_values)
    
    # 각 selection별 전체 평균 계산 및 그리기
    x_labels = ['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7']
    for j, (selection, values) in enumerate(selection_averages.items()):
        if values:  # 데이터가 있는 경우에만
            overall_avg = np.mean(values, axis=0)
            ax.plot(x_labels, overall_avg, marker='o', label=selection, alpha=0.4)
    
    ax.set_xlabel('Objective Functions')
    ax.set_ylabel('Fitness Value')
    ax.legend()

# 사용하지 않는 subplot 숨기기
for i in range(num_problems + 1, len(axes)):
    axes[i].set_visible(False)

plt.tight_layout()
plt.show()

In [1]:
!pip install pycparser


[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
from pycparser import c_parser, c_ast, parse_file
import pycparser

class LoopNestingVisitor(c_ast.NodeVisitor):
    """AST를 방문하며 루프 중첩 깊이를 계산"""
    
    def __init__(self):
        self.max_nesting = 0
        self.current_nesting = 0
    
    def visit_For(self, node):
        """For 루프 방문"""
        self.current_nesting += 1
        self.max_nesting = max(self.max_nesting, self.current_nesting)
        # 자식 노드 방문
        self.generic_visit(node)
        self.current_nesting -= 1
    
    def visit_While(self, node):
        """While 루프 방문"""
        self.current_nesting += 1
        self.max_nesting = max(self.max_nesting, self.current_nesting)
        self.generic_visit(node)
        self.current_nesting -= 1
    
    def visit_DoWhile(self, node):
        """Do-While 루프 방문"""
        self.current_nesting += 1
        self.max_nesting = max(self.max_nesting, self.current_nesting)
        self.generic_visit(node)
        self.current_nesting -= 1

def count_loop_nesting(c_code):
    """
    C 코드의 최대 루프 중첩 깊이를 계산
    
    Args:
        c_code: C 소스 코드 문자열
    
    Returns:
        int: 최대 루프 중첩 깊이
    """
    try:
        # pycparser는 전처리된 코드가 필요하므로 간단한 전처리
        # 실제로는 fake_libc_include를 사용하거나 전처리 필요
        parser = c_parser.CParser()
        
        # 간단한 wrapper 추가 (함수가 아닌 경우)
        if 'int main' not in c_code and 'void ' not in c_code:
            c_code = f"void temp_func() {{ {c_code} }}"
        
        ast = parser.parse(c_code)
        
        visitor = LoopNestingVisitor()
        visitor.visit(ast)
        
        return visitor.max_nesting
    
    except Exception as e:
        print(f"Parsing error: {e}")
        return 0

# 사용 예시
c_code1 = """
int main() {
    for (int i = 0; i < 10; i++) {
        for (int j = 0; j < 10; j++) {
            printf("%d", i + j);
        }
    }
    return 0;
}
"""

c_code2 = """
int main() {
    for (int i = 0; i < 10; i++) {
        for (int j = 0; j < 10; j++) {
            for (int k = 0; k < 10; k++) {
                printf("%d", i + j + k);
            }
        }
    }
    return 0;
}
"""

print(f"Code1 nesting depth: {count_loop_nesting(c_code1)}")  # 2
print(f"Code2 nesting depth: {count_loop_nesting(c_code2)}")  # 3

Code1 nesting depth: 2
Code2 nesting depth: 3


In [ ]:
from src.utils import Loader

dataset_path = 'data/problemID_2/dataset.json'
loader = Loader()
problemId, description, buggys, references, testcases = loader.run(dataset_path)
prompts = f"""{description}

[reference code]
{references[0].code}
[buggy code]
{buggys[0].code}

please fix the buggy code and return the correct code.
"""

# print(prompts)
# print(testcases)

In [ ]:
import re
from transformers import AutoTokenizer
from src.genetic import Fitness
from src.execution import Tester
from src.llms import Ollama
from pydantic import BaseModel
from tqdm.notebook import tqdm

SYSTEM_PROMPT = '''# Role

You are an expert {language} programming tutor who helps to fix buggy program.

# Task

You will be given a 'Problem Description', a set of 'Test Cases', a 'Buggy Program', a 'Reference Program' and their test results as Inputs.
Generate the 'Fixed Program' by repairing the 'Buggy Program' according to the following priority guidelines:
{guidelines}

Outputs should be a 'Fixed Program' as str.
'''

USER_PROMPT = """# Inputs

## Problem Description
{description}

## Buggy Program
{buggy_program}
### Buggy Test Results
{buggy_results}

## Reference Program
{reference_program}
### Reference Test Results
{reference_results}
"""

ollama = Ollama()
fitness = Fitness()
Tester.init_globals(testcases, timelimit=1)
guidelines = fitness.guidelines.values()
tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-hf", use_fast=False)

class OutFormat(BaseModel):
    fixed_program: str

def post_process(text: str) -> str | None:
    if text.startswith("```") and text.endswith("```"):
        m = re.search(r"```(?:[a-zA-Z0-9_+-]+)?\n(.*?)```", text, flags=re.DOTALL)
        return m.group(1).strip() if m else None
    return text.strip()

avg_tokens = []
out_tokens = []
fixed_programs = []
for buggy in tqdm(buggys):
    refer = references.get_prog_by_id(buggy.id)
    
    b_results = Tester.run(buggy)
    b_passed, b_failed = Tester.tests_split(b_results)
    buggy_results = f"Passed Test Cases IDs: {sorted([tc.id for tc in b_passed])}  \n"
    buggy_results += f"Failed Test Cases IDs: {sorted([tc.id for tc in b_failed])}\n"     
    
    r_results = Tester.run(refer)
    r_passed, r_failed = Tester.tests_split(r_results)
    reference_results = f"Passed Test Cases IDs: {sorted([tc.id for tc in r_passed])}  \n"
    reference_results += f"Failed Test Cases IDs: {sorted([tc.id for tc in r_failed])}\n"
    
    # System Prompt
    system = SYSTEM_PROMPT.format(
        language=buggy.ext,
        guidelines=guidelines
    )
    
    # User Prompt
    user = USER_PROMPT.format(
        description=description,
        buggy_program=f"```{buggy.ext}\n{buggy.code}\n```",
        buggy_results=buggy_results,
        reference_program=f"```{refer.ext}\n{refer.code}\n```",
        reference_results=reference_results,
    )
    
    prompt = system + "\n" + user
    tokens = tokenizer.tokenize(prompt)
    avg_tokens.append(len(tokens))
    
    # response = await ollama.run(system=system, user=user, format=OutFormat)
    # if response is not None:
    #     fixed_program = post_process(response.fixed_program)
    #     out_tok = tokenizer.tokenize(fixed_program)
    #     fixed_programs.append(fixed_program)
    # else:
    #     out_tok = []
    # out_tokens.append(len(out_tok))

print("Min tokens:", min(avg_tokens))
print("Average tokens:", sum(avg_tokens) / len(references))
print("Max tokens:", max(avg_tokens))

# print("Min output tokens:", min(out_tokens))
# print("Average output tokens:", sum(out_tokens) / len(references))
# print("Max output tokens:", max(out_tokens))

  0%|          | 0/123 [00:00<?, ?it/s]

Min tokens: 1065
Average tokens: 1383.2032520325204
Max tokens: 2647
